In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 

In [2]:
from os import path
train_path= path.join('Documents','MasterIA','Python para la IA','titanic','train.csv')
test_path= path.join('Documents','MasterIA','Python para la IA','titanic','test.csv')

In [4]:
train_df = pd.read_csv('C:/Users/User/Documents/MasterIA/Python para la IA/titanic/train.csv')
test_df = pd.read_csv('C:/Users/User/Documents/MasterIA/Python para la IA/titanic/test.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/User/Documents/MasterIA/Python para la IA/titanic/train.csv'

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
print(train_df.shape)
print(test_df.shape)    #revisamos la forma del DF y vemos la proporcion entre train y test

In [ ]:
display(train_df.describe(include='all'))  # .describe() para ver las metricas de las diferentes variables.

In [ ]:
train_df.info()      #ver missing values

In [ ]:
sns.barplot(x='Embarked', y='Survived' , hue = 'Sex' , data= train_df)  #Embarque influye en supervivencia --> NO, Sex SI

In [ ]:
sns.pointplot(data= train_df , x='Pclass',y= 'Survived',hue = 'Sex') # Grafico de Clase frente a supervivencia

In [ ]:
# Categorizar datos --> Age
# Listo los valores unicos de edad

so = list(train_df.Age.unique())
so.sort()                              
print(so)

In [ ]:
train_df.hist(column='Age')
plt.show()

In [ ]:
#Funcion que categoriza la edad.
#Si valor es de la forma 0.XX multiplica por 100 para corregir el dato
# Valores vacios sustituidos por -0.5 par despues asignar categoria a negativos


def cat_age(data):
    
    data.loc[(data['Age'] > 0) & (data['Age'] <1) , 'Age']*100
    
    data['Age']=data['Age'].fillna(-0.5)
    
    bins = (-1 ,0 , 12 ,20, 35 , 60 , 110)               #intervalos en los que quiero dividir los datos
    cat_names = ('Unknown', 'Child', 'Teenager' , 'Young Adult', 'Adult', 'Elder') # ñista de categorias en los que estan repartidos
    
    categories = pd.cut(data['Age'], bins , labels = cat_names)
    data['Age']= categories
    return

In [ ]:
cat_age(train_df)


In [ ]:
train_df.sample(10)

In [ ]:
cat_age(test_df)
test_df.sample(5)

In [ ]:
sns.barplot(x='Age',y= 'Survived', data = train_df , units=True)

In [ ]:
train_df.Age.value_counts()

In [ ]:
train_df['Cabin'].sample(10)

In [ ]:
def extract_cabin(data): 
    
    data['Cabin'] = data['Cabin'].fillna('N')            #Sustituyo los NaN por 'N'
    
    data['Cabin'] = data['Cabin'].apply(lambda x : x[0]) # Modificar la columna con funcion lambdapata quedarme con el primer
                                                          #argumento del dato

In [ ]:
extract_cabin(train_df)
extract_cabin(test_df)

In [ ]:
train_df.Cabin.sample(10)

In [ ]:
sns.countplot(x='Cabin', hue= 'Survived' , data= train_df)  # Cabin parece que no influye

In [ ]:
#filtered_df = train_df[train_df['Cabin'] != 'N']


In [ ]:
#sns.countplot(x='Cabin', hue= 'Survived' , data= filtered_df)

In [ ]:
def cat_fare(data):
    data['Fare'] = data['Fare'].fillna(0)    #data['Fare'].mean()
    cat_names= ['1st','2nd','3rd','4th','5th']                      #Categorizar Fare , sistituyendo por 0 los NaN
                                                                    #Mismo proceso que con Age
    data['Fare'] = pd.qcut(data['Fare'], 5 , labels = cat_names)

In [ ]:
cat_fare(train_df)


In [ ]:
cat_fare(test_df)

In [ ]:
test_df.sample(5)

In [ ]:
def drop_columns(data):
    data = data.drop(['PassengerId','Name','Ticket', 'Embarked'], axis=1, inplace= True)

In [ ]:
drop_columns(train_df)
train_df.sample(5)

In [ ]:
drop_columns(test_df)
test_df.sample(5)

In [ ]:
sns.barplot(x='Age', y='Survived', hue='Sex', data=train_df)

In [ ]:
sns.barplot(x='Fare', y='Survived', hue='Sex', data=train_df)

In [ ]:
train_df.head()

***Datos Preparados para entrenarse, falta transformar variables categoricas para trabajar con algoritmos de clasificacion***

In [ ]:
from sklearn import preprocessing

In [ ]:
def encode_features(train, test, features):
    
    df_combined=pd.concat([train[features], test[features]])
    
    for f in features:
        le=preprocessing.LabelEncoder()
        le=le.fit(df_combined[f])
        
        train[f] = le.transform(train[f])
        test[f] = le.transform(test[f])

In [ ]:
features= ['Sex','Age','Fare','Cabin']

encode_features(train_df, test_df, features)

train_df.head()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = train_df.drop(['Survived'], axis=1)  #Dataset con las varibles independientes
Y = train_df['Survived']                 #Dataset con la dependiente (Sobrevivir)

validation_size= 0.15        #Tamaño % del data Set para hacer la validacion una vez entrenado

seed = np.random.randint(1000)  #semilla aleatoria

X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size = validation_size, random_state=seed)

train_test_split(X, Y, test_size = validation_size, random_state=seed)

*Esto devuelve 4 DataSets*

**85% del de entrenamiento con las val dependientes       X_train**

**85% del de entrenamiento con la varible 'Survived       Y_train '**


**15% del de entrenamiento para validar con las variables dependientes    X_validation**

**15% del de entrenamiento para validar con la variable 'Survived'        Y_validation**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

In [ ]:
model=DecisionTreeClassifier()
model.fit(X_train, Y_train)

In [ ]:
fig = plt.figure(figsize=(15, 10))
_ = tree.plot_tree(model, feature_names=X_train.columns,class_names=['Not Survived', 'Survived'], filled=True)

plt.show()

In [ ]:
from sklearn.metrics import accuracy_score

prediction= model.predict(X_validation)
print(prediction)
print(accuracy_score(prediction, Y_validation))     #Puntuacion de precision

In [ ]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(Y_validation, prediction))    #Matriz de confusion


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(Y_validation, prediction))


In [ ]:
n_iters = 10

results= []

for i in range(n_iters):
    X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size = validation_size, random_state=seed)
    
    model=DecisionTreeClassifier()
    model.fit(X_train, Y_train)
    
    prediction= model.predict(X_validation)
    results.append(accuracy_score(prediction, Y_validation))
    
print(results)
print(sum(results)/len(results))